In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

# proFSA
import fsa.load as load
import fsa.spatial as spatial
import fsa.sequence as seq
import fsa.elastic as elastic

In [2]:
# Input
rel_pdb, def_pdb = '6FKI', '6FKF'

# Load
rel_pps = load.single_structure(rel_pdb)
def_pps = load.single_structure(def_pdb)

In [3]:
# Align
uni_ids = [None] * len(rel_pps)
my_ref_seqs = seq.generate_reference(rel_pps, uni_ids)
rel_idx = seq.align(rel_pps, my_ref_seqs)
def_idx = seq.align(def_pps, my_ref_seqs)
rel_dict = seq.aligned_dict(rel_pps, rel_idx)
def_dict = seq.aligned_dict(def_pps, def_idx)
com_res = seq.common(rel_dict, def_dict)

In [4]:
# Coordinates
rel_xyz, rel_labels = load.coordinates(rel_pps,
                                       com_res,
                                       rel_dict)
def_xyz, def_labels = load.coordinates(def_pps,
                                       com_res,
                                       def_dict)

In [5]:
# Strain prepare
weights = elastic.linear_weights_fast([rel_xyz, def_xyz],
                                      parameters=[10.,10.1])

F = elastic.deformation_gradient_fast(weights, rel_xyz, def_xyz)


In [6]:
# Strain results
rot_angle, rot_axis = elastic.rotations(F)
rot_axis = spatial.vector_to_polar(rel_xyz, rot_axis)
_, gam_n = elastic.lagrange_strain(F)
stretches, stretch_axis = elastic.principal_stretches_from_g(gam_n)

In [7]:
# Save to PDB
strain_cluster_dict = {}
for label, st in zip(rel_labels, np.abs(stretches[:, 0])):
    strain_cluster_dict[label[0]] = st

load.save_bfactor(name=rel_pdb,
                  bfact=strain_cluster_dict,
                  ext='_st_quant_L0')

In [8]:
# Create dictionaries and files with principal stretches and rotation angles
var_values = [np.abs(stretches[:, 0]), 
              np.abs(stretches[:, 1]), 
              np.abs(stretches[:, 2]), 
              rot_angle]

var_names = ["lambda_1", 
             "lambda_2", 
             "lambda_3", 
             "rotation_angle"]

for name, value in zip(var_names, var_values):
    filename = rel_pdb + "-" + def_pdb + "-" + name
    var_cluster_dict = {}
    
    with open(filename + ".txt", "w") as f:
        for label, var in zip(rel_labels, value):
            var_cluster_dict[label[0]] = var
            f.write(str(label[0]) + ": " + str(var) + "\n")